In [3]:
library(ChIPseeker)
library(clusterProfiler)
library(GenomicFeatures)
library(AnnotationDbi)
library(TxDb.Mmusculus.UCSC.mm10.knownGene)
library(ReactomePA)
library(tidyverse)

txdb <- TxDb.Mmusculus.UCSC.mm10.knownGene
promoter <- getPromoters(TxDb=txdb, upstream=5000, downstream=5000)

if (! exists("code.dir")) {
    code.dir <- getwd()
    out.dir <- gsub("codes_local", "3_peak_annotations", code.dir)
}

In [11]:
input.dir <- '/media/pipkin/Rocket2/T_Cell_ChIP/202012_ChIP/2_MACS2'
input.peak.files <- list.files(path=input.dir, patter='narrowPeak.chr.bed',full.name=TRUE)

bp.range <- c(-5000, 5000)
attempted <- c()

In [12]:
input.peak.files <- input.peak.files[95:97]
input.peak.files

[1] "/media/pipkin/Rocket2/T_Cell_ChIP/202012_ChIP/2_MACS2/2021_GoldrathLab_Brd4___BRD4_MP-CD8_peaks.narrowPeak.chr.bed" 
[2] "/media/pipkin/Rocket2/T_Cell_ChIP/202012_ChIP/2_MACS2/2021_GoldrathLab_Brd4___BRD4_NAV-CD8_peaks.narrowPeak.chr.bed"
[3] "/media/pipkin/Rocket2/T_Cell_ChIP/202012_ChIP/2_MACS2/2021_GoldrathLab_Brd4___BRD4_TE-CD8_peaks.narrowPeak.chr.bed"

In [13]:
for (file.i in input.peak.files){
    if (!(file.i %in% attempted)) {
        attempted <- c(attempted, file.i)
        i <- gsub("_peaks.narrowPeak.chr.bed","", basename(file.i))
        print("-----------")
        print(i)
        peak.i <- readPeakFile(file.i)
        
        if (length(peak.i@ranges@start) > 200) {
            #----- Coverage
            covplot.i <- covplot(peak.i, weightCol="V5")
            plot.name.i <- file.path(out.dir, paste(i, "covPlot.png", sep="_"))
            ggsave(plot.name.i, covplot.i, device='png', scale=2)

            #----- TagMatrix
            tagMatrix.i <- getTagMatrix(peak.i, windows=promoter)

            plot.name.i <- file.path(out.dir, paste(i, "tagMatrix.jpg", sep="_"))
            jpeg(filename=plot.name.i,width=400,height=600)
            tagHeatmap(tagMatrix.i, xlim=bp.range, color=NULL)
            dev.off()

            #----- Average Prof
            #avgprof.plot.i <- plotAvgProf(tagMatrix.i, xlim=bp.range, conf=0.95, resample = 1000)
            #plot.name.i <- file.path(out.dir, paste(i, "avgProfPlot.png", sep="_"))
            #ggsave(plot.name.i, avgprof.plot.i, device='png', scale=1)

            #----- Peak Annotateion
            peakAnno <- annotatePeak(file.i, tssRegion=bp.range, TxDb=txdb, annoDb="org.Mm.eg.db")
            peakAnno.df <- as.data.frame(peakAnno)
            df.name.i <- file.path(out.dir, paste(i, "annoDf.csv", sep="_"))
            write_csv(peakAnno.df, df.name.i)

            #----- Annotation Pie Chart
            plot.name.i <- file.path(out.dir, paste(i, "annoPiePlot.jpg", sep="_"))
            jpeg(filename=plot.name.i,width=480,height=480)
            plotAnnoPie(peakAnno)
            dev.off()

            #----- Pathway Analysis
            pathway.i <- enrichPathway(peakAnno.df$geneId, organism='mouse')
            df.name.i <- file.path(out.dir, paste(i, "pathway.csv", sep="_"))
            write_csv(as.data.frame(pathway.i), df.name.i)

            pathway.plot.i <- dotplot(pathway.i)
            plot.name.i <- file.path(out.dir, paste(i, "pathwayDotPlot.png", sep="_"))
            ggsave(plot.name.i, pathway.plot.i, device='png', scale=2)
        }    
    }
}

[1] "-----------"
[1] "2021_GoldrathLab_Brd4___BRD4_MP-CD8"


Saving 13.3 x 13.3 in image



>> loading peak file...				 2021-01-22 01:21:52 AM 
>> preparing features information...		 2021-01-22 01:21:52 AM 
>> identifying nearest features...		 2021-01-22 01:21:53 AM 
>> calculating distance from peak to TSS...	 2021-01-22 01:21:53 AM 
>> assigning genomic annotation...		 2021-01-22 01:21:53 AM 
>> adding gene annotation...			 2021-01-22 01:22:05 AM 


Loading required package: org.Mm.eg.db



'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2021-01-22 01:22:06 AM 
>> done...					 2021-01-22 01:22:06 AM 


...

Saving 13.3 x 13.3 in image



>> loading peak file...				 2021-01-22 01:23:39 AM 
>> preparing features information...		 2021-01-22 01:23:39 AM 
>> identifying nearest features...		 2021-01-22 01:23:39 AM 
>> calculating distance from peak to TSS...	 2021-01-22 01:23:39 AM 
>> assigning genomic annotation...		 2021-01-22 01:23:39 AM 
>> adding gene annotation...			 2021-01-22 01:23:42 AM 


'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2021-01-22 01:23:42 AM 
>> done...					 2021-01-22 01:23:42 AM 


Saving 13.3 x 13.3 in image

